# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4877, uptime 0:00:17
mini_beamline                    RUNNING   pid 4878, uptime 0:00:17
random_walk                      RUNNING   pid 4879, uptime 0:00:17
random_walk_horiz                RUNNING   pid 4880, uptime 0:00:17
random_walk_vert                 RUNNING   pid 4881, uptime 0:00:17
simple                           RUNNING   pid 4882, uptime 0:00:17
thermo_sim                       RUNNING   pid 4883, uptime 0:00:17
trigger_with_pc                  RUNNING   pid 4884, uptime 0:00:17


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 1     Time: 2020-05-14 19:01:14
Persistent Unique Scan ID: '50295a6f-89a5-4459-bb9f-75479646250d'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 19:01:14.9 |     -1.000 |      95241 |


|         2 | 19:01:15.4 |     -0.500 |     102102 |


|         3 | 19:01:16.1 |      0.000 |     105398 |


|         4 | 19:01:16.6 |      0.500 |     102906 |


|         5 | 19:01:17.1 |      1.000 |      98236 |
+-----------+------------+------------+------------+
generator scan ['50295a6f'] (scan num: 1)





('50295a6f-89a5-4459-bb9f-75479646250d',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/50295a6f-89a5-4459-bb9f-75479646250d-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 2     Time: 2020-05-14 19:01:17
Persistent Unique Scan ID: 'd131552c-1f23-4ffe-862b-a631d96cb127'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 19:01:17.6 |          -1 | 1073367708 |


|         2 | 19:01:17.7 |          -0 | 1068317266 |


|         3 | 19:01:17.8 |           0 | 1060152851 |


|         4 | 19:01:17.9 |           0 | 1056446963 |


|         5 | 19:01:18.0 |           1 | 1054203752 |


+-----------+------------+-------------+------------+
generator scan ['d131552c'] (scan num: 2)





('d131552c-1f23-4ffe-862b-a631d96cb127',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-14 19:01:18
Persistent Unique Scan ID: '2433abe8-6ad7-460d-b635-a8400f6ddce5'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 19:01:18.6 | 1056536262 |


|         2 | 19:01:19.5 | 1111002143 |


|         3 | 19:01:20.5 | 1127033144 |


...

|        28 | 19:01:45.5 | 1071248678 |


|        29 | 19:01:46.5 | 1059647494 |


|        30 | 19:01:47.5 | 1114341877 |


+-----------+------------+------------+
generator count ['2433abe8'] (scan num: 3)





('2433abe8-6ad7-460d-b635-a8400f6ddce5',)